# 📚 מדריך מקיף: עבודה עם מבני נתונים מקוננים ב-Polars

## תוכן עניינים

1. [מבוא ודרישות טכניות](#מבוא)
2. [יצירת רשימות (Lists)](#יצירת-רשימות)
3. [צבירת אלמנטים ברשימות](#צבירת-אלמנטים)
4. [גישה ובחירת אלמנטים](#גישה-ובחירה)
5. [החלת לוגיקה על אלמנטים](#החלת-לוגיקה)
6. [עבודה עם Structs ו-JSON](#עבודה-עם-structs)
7. [תרגילים ואתגרים](#תרגילים)
8. [סיכום ומשאבים נוספים](#סיכום)

---

<a id="מבוא"></a>
## 🎯 מבוא

### מהם מבני נתונים מקוננים?

מבני נתונים מקוננים הם מבנים שמכילים בתוכם מבנים אחרים. בPolars, אנחנו עובדים בעיקר עם:

- **רשימות (Lists)**: עמודות שכל תא בהן מכיל רשימה של ערכים
- **מבנים (Structs)**: עמודות שכל תא בהן מכיל מספר שדות (כמו dictionary)
- **JSON**: נתונים היררכיים שניתן לפרק ולעבד

### למה זה חשוב?

✅ **ביצועים גבוהים**: Polars מטפל ביעילות במבנים מורכבים  
✅ **גמישות**: ניתן לאחסן מידע מורכב בעמודה אחת  
✅ **שאילתות מתקדמות**: פעולות חזקות על נתונים היררכיים  

### דרישות טכניות

In [ ]:
# התקנת Polars (אם עדיין לא מותקן)
# !pip install polars

# ייבוא הספרייה
import polars as pl
import warnings
warnings.filterwarnings('ignore')

# הצגת גרסה
print(f"🐻 Polars Version: {pl.__version__}")

### טעינת הנתונים

נעבוד עם מערך נתונים של סרטוני YouTube שהיו פופולריים (trending) בארה"ב.

In [ ]:
# טעינת הנתונים
# שים לב: יש להתאים את הנתיב לקובץ ה-CSV שלך
df = pl.read_csv("../data/us_videos.csv", try_parse_dates=True)

# המרת עמודת התאריכים לפורמט נכון
df = df.with_columns(
    pl.col("trending_date").str.to_date(format="%y.%d.%m")
)

# הצגת מידע כללי
print(f"📊 מספר שורות: {df.height:,}")
print(f"📊 מספר עמודות: {df.width}")
print(f"\n🔍 5 השורות הראשונות:")
df.head()

In [ ]:
# בואו נראה את מבנה הנתונים
df.glimpse(max_items_per_column=2)

### 💡 **טיפ חשוב**

> לפני שמתחילים לעבוד עם נתונים מקוננים, חשוב להבין את מבנה הנתונים המקורי. השתמשו ב-`glimpse()` ו-`describe()` כדי לחקור את הנתונים!

---
<a id="יצירת-רשימות"></a>
## 📝 1. יצירת רשימות (Creating Lists)

### מהי רשימה ב-Polars?

רשימה (List) היא עמודה שבה כל תא מכיל **וקטור של ערכים**. זה שונה מעמודה רגילה שבה כל תא מכיל **ערך בודד**.

### שלוש דרכים ליצור רשימות:

1. **פיצול מחרוזות** - `str.split()`
2. **צבירת ערכים** - `group_by()` + `agg()`
3. **איחוד עמודות** - `concat_list()`

### 🔹 דרך 1: פיצול מחרוזות (String Split)

בעמודת `tags` יש תגיות מופרדות בתו `|`. בואו נפצל אותן לרשימה:

In [ ]:
# פיצול עמודת התגיות
result = df.select(
    'tags',
    pl.col('tags').str.split('|').alias('tags_list')
).head()

print("🔍 התוצאה:")
result

In [ ]:
# בואו נראה את הטיפוס של העמודה החדשה
print("📌 טיפוס העמודה המקורית:", df.select('tags').dtypes[0])
print("📌 טיפוס העמודה החדשה:", result.select('tags_list').dtypes[0])

### 💡 **הסבר**

- `str.split('|')` מפצל מחרוזת לפי התו `|`
- התוצאה היא עמודה מטיפוס `list[str]` (רשימה של מחרוזות)
- כל תא מכיל כעת **רשימה** במקום מחרוזת בודדת

### 🔹 דרך 2: צבירת ערכים (Aggregation)

כאשר אנחנו מבצעים `group_by()`, אפשר לצבור ערכים לתוך רשימות:

In [ ]:
# צבירת כל מזהי הסרטונים לפי תאריך
videos_by_date = (
    df
    .group_by('trending_date')
    .agg(pl.col('video_id'))
    .sort('trending_date', descending=True)
).head()

print("📅 סרטונים לפי תאריך:")
videos_by_date

In [ ]:
# כמה סרטונים יש בכל תאריך?
videos_by_date.with_columns(
    pl.col('video_id').list.len().alias('num_videos')
)

### 💡 **הסבר**

- `group_by('trending_date')` מקבץ שורות לפי תאריך
- `agg(pl.col('video_id'))` אוסף את כל מזהי הסרטונים **לרשימה אחת**
- כל תאריך מקבל רשימה של כל הסרטונים שהיו טרנדיים באותו היום

### 🔹 דרך 3: איחוד עמודות (Concatenating Columns)

נוכל לקחת מספר עמודות ולאחד אותן לרשימה אחת:

In [ ]:
# איחוד מדדי מעורבות לרשימה אחת
engagement_list = df.select(
    pl.concat_list(
        pl.col('views'),
        pl.col('likes'),
        pl.col('dislikes'),
        pl.col('comment_count')
    ).alias('engagement_metrics')
)

print("📊 מדדי מעורבות:")
print(engagement_list)

In [ ]:
# בואו נראה איך זה נראה עם שמות העמודות
df.select(
    'title',
    'views',
    'likes', 
    'dislikes',
    'comment_count',
    pl.concat_list('views', 'likes', 'dislikes', 'comment_count').alias('all_metrics')
).head(3)

### 💡 **הסבר**

- `concat_list()` מאחד ערכים מעמודות שונות **לרשימה בתא אחד**
- השורה הראשונה: `[748374, 57527, 2966, 15954]`
  - 748,374 צפיות
  - 57,527 לייקים
  - 2,966 דיסלייקים
  - 15,954 תגובות

### 🎓 תרגיל 1: יצירת רשימות

**משימה**: צור רשימה שמכילה את שם הערוץ, הכותרת ומספר הצפיות לכל סרטון.

In [ ]:
# כתוב את הקוד שלך כאן:
# רמז: השתמש ב-concat_list עם העמודות הנכונות
df.select(
    'title',
    'views',
    'channel_title', 
    pl.concat_list('title', 'channel_title', 'views').alias('all_metrics')
).head(3)


### 📌 רשימות מקוננות (Nested Lists)

Polars תומך גם ברשימות של רשימות!

In [ ]:
# יצירת רשימה מקוננת
nested_df = pl.DataFrame({
    'nested_list': [
        [[1,2,3], [4,5,6], [7,8,9], [10,11,12]], 
        [[1,2,3], [4,5,6], [7,8,9], [10,11,12]]
    ]
})

print("🔢 רשימה מקוננת:")
print(f"טיפוס: {nested_df.dtypes[0]}")
nested_df

### ⚠️ **שים לב**: טיפוסים מעורבים

אם יש ערכים מטיפוסים שונים, Polars ינסה להמיר הכל לטיפוס משותף:

In [ ]:
# רשימה עם טיפוסים מעורבים
mixed_df = pl.DataFrame({
    'mixed_list': [
        [['a',2,3], [4,5,6], [7,8,9]], 
        [[1,2,3], [4,5,6], [7,8,9]]
    ]
}, strict=False)  # strict=False מאפשר המרות אוטומטיות

print("🔀 רשימה מעורבת:")
print(f"טיפוס: {mixed_df.dtypes[0]}")
mixed_df

---
<a id="צבירת-אלמנטים"></a>
## 🧮 2. צבירת אלמנטים ברשימות (Aggregating Elements)

### מהי צבירה?

**צבירה** (Aggregation) היא פעולה שלוקחת רשימה של ערכים ומחזירה **ערך בודד**.

### פעולות צבירה נפוצות:

| פעולה | תיאור | דוגמה |
|-------|--------|-------|
| `list.min()` | מינימום | `[3,1,4,1,5] → 1` |
| `list.max()` | מקסימום | `[3,1,4,1,5] → 5` |
| `list.mean()` | ממוצע | `[3,1,4,1,5] → 2.8` |
| `list.sum()` | סכום | `[3,1,4,1,5] → 14` |
| `list.len()` | אורך | `[3,1,4,1,5] → 5` |
| `list.join()` | איחוד למחרוזת | `['a','b','c'] → 'a:b:c'` |

### דוגמה מעשית: ניתוח סטטיסטי של צפיות

In [ ]:
# קודם נצבור את הנתונים לרשימות
agg_df = (
    df
    .group_by('trending_date')
    .agg('views', 'likes', 'dislikes', 'comment_count')
    .sort('trending_date', descending=True)
)

print("📊 נתונים מצוברים:")
print(agg_df.shape)
agg_df.head()

In [ ]:
# עכשיו נבצע פעולות צבירה על הרשימות
stats_df = agg_df.select(
    'trending_date',
    # סטטיסטיקות על צפיות
    pl.col('views').list.min().alias('min_views'),
    pl.col('views').list.max().alias('max_views'),
    pl.col('views').list.mean().alias('avg_views'),
    
    # סטטיסטיקות על לייקים
    pl.col('likes').list.max().alias('max_likes'),
    
    # סטטיסטיקות על דיסלייקים
    pl.col('dislikes').list.mean().alias('avg_dislikes'),
    
    # סכום תגובות
    pl.col('comment_count').list.sum().alias('total_comments')
).head()

print("📈 סטטיסטיקות מצפיות:")
for col in ['min_views', 'max_views', 'avg_views', 'max_likes', 'avg_dislikes', 'total_comments']:
    row = stats_df.sort(col, descending=True).row(0, named=True)
    print(f"{col}: {row[col]:,.0f} ({row['trending_date']})")
stats_df

### 💡 **פירוש התוצאות**

בתאריך 2018-06-14:
- **מינימום צפיות**: 189,038 (הסרטון הכי פחות נצפה)
- **מקסימום צפיות**: סרטון עם הכי הרבה צפיות
- **ממוצע צפיות**: הממוצע של כל הסרטונים
- **סה"ך תגובות**: סך כל התגובות על כל הסרטונים

### 🔹 איחוד רשימות למחרוזת (Join)

פעולת `join` מאחדת אלמנטים לתוך מחרוזת אחת:

In [ ]:
# איחוד שמות ערוצים
channel_summary = (
    df
    .group_by('trending_date')
    .agg(pl.col('channel_title'))
    .with_columns(
        pl.col('channel_title').list.join(', ').alias('all_channels')
    )
    .select('trending_date', 'all_channels')
    .sort('trending_date', descending=True)
).head(3)

print("📺 ערוצים לפי תאריך:")
channel_summary

### 🔹 ספירת אלמנטים (Length)

כמה אלמנטים יש ברשימה?

In [ ]:
# כמה סרטונים היו טרנדיים בכל יום?
video_counts = agg_df.select(
    'trending_date',
    pl.col('views').list.len().alias('num_trending_videos')
).head(10)

print("📊 מספר סרטונים טרנדיים ליום:")
video_counts

### 🎓 תרגיל 2: צבירת נתונים

**משימה**: 
1. קבץ את הנתונים לפי `category_id`
2. מצא את מספר הצפיות הממוצע, המינימום והמקסימום לכל קטגוריה
3. מיין לפי ממוצע הצפיות (יורד)

In [ ]:
res = (
    df
    .group_by('category_id')
    .agg([
        pl.col('views').mean().alias('avg_views'),
        pl.col('views').min().alias('min_views'),
        pl.col('views').max().alias('max_views')
    ])
    .head()
)

res

---
<a id="גישה-ובחירה"></a>
## 🎯 3. גישה ובחירת אלמנטים (Accessing and Selecting Elements)

### איך ניגשים לאלמנטים ברשימה?

Polars מספק מגוון פעולות לבחירת אלמנטים ספציפיים מרשימות.

### טבלת פעולות גישה:

| פעולה | תיאור | דוגמה |
|-------|--------|-------|
| `list.first()` | האלמנט הראשון | `[1,2,3] → 1` |
| `list.last()` | האלמנט האחרון | `[1,2,3] → 3` |
| `list.get(n)` | אלמנט במיקום n | `[1,2,3].get(1) → 2` |
| `list.head(n)` | n אלמנטים ראשונים | `[1,2,3,4].head(2) → [1,2]` |
| `list.tail(n)` | n אלמנטים אחרונים | `[1,2,3,4].tail(2) → [3,4]` |
| `list.slice(offset, length)` | חיתוך מותאם | `[1,2,3,4].slice(1,2) → [2,3]` |

### הכנת הנתונים

In [ ]:
# נצור DataFrame עם רשימות של תאריכים לכל ערוץ
trending_dates_by_channel = (
    df
    .group_by('channel_title')
    .agg('trending_date')
    .with_columns(
        # מיון התאריכים בתוך הרשימה
        pl.col('trending_date').list.sort()
    )
)

print("📺 תאריכים טרנדיים לפי ערוץ:")
trending_dates_by_channel.head()

### 🔹 אלמנטים ראשונים ואחרונים

In [ ]:
# התאריך הראשון והאחרון שבו הערוץ היה טרנדי
first_last = trending_dates_by_channel.with_columns(
    pl.col('trending_date').list.first().alias('first_trending_date'),
    pl.col('trending_date').list.last().alias('last_trending_date')
).head()

print("📅 תאריכים ראשונים ואחרונים:")
first_last

### 🔹 גישה לאלמנט במיקום ספציפי

In [ ]:
# האלמנט ה-8 (אינדקס 7)
eighth_element = trending_dates_by_channel.with_columns(
    pl.col('trending_date').list.get(7, null_on_oob=True).alias('8th_trending_date')
).head()

print("🔢 האלמנט השמיני:")
eighth_element

### 💡 **הסבר**: `null_on_oob=True`

- **OOB** = Out Of Bounds (מחוץ לגבולות)
- אם הרשימה קצרה מדי, החזר `null` במקום שגיאה
- בלי זה, תתקבל שגיאה אם הרשימה קצרה מ-8 אלמנטים

### 🔹 בחירת מספר אלמנטים

In [ ]:
# 5 ראשונים ו-10 אחרונים
head_tail = trending_dates_by_channel.with_columns(
    pl.col('trending_date').list.head(5).alias('first_5_dates'),
    pl.col('trending_date').list.tail(10).alias('last_10_dates')
).head(3)

print("📊 ראשונים ואחרונים:")
head_tail

### 🔹 חיתוך מתקדם (Slicing)

חיתוך דומה ל-Python slicing, אבל עם תחביר שונה:

In [ ]:
# דוגמאות לחיתוך
slicing_examples = trending_dates_by_channel.select(
    'trending_date',
    # 2 ראשונים (התחל מ-0, קח 2)
    pl.col('trending_date').list.slice(0, 2).alias('first_2'),
    
    # האלמנט השלישי מהסוף (התחל מ--3, קח 1)
    pl.col('trending_date').list.slice(-3, 1).alias('3rd_from_end'),
    
    # מהאלמנט ה-8 עד הסוף
    pl.col('trending_date').list.slice(7).alias('from_8th_onward')
).head()

print("✂️ דוגמאות חיתוך:")
slicing_examples

### 💡 **תחביר slice**

```python
list.slice(offset, length)
```

- **offset**: נקודת ההתחלה (יכול להיות שלילי לספירה מהסוף)
- **length**: כמה אלמנטים לקחת (אופציונלי, ברירת מחדל עד הסוף)

### 🔹 בחירת אלמנטים מרובים (Gather)

`gather` מאפשר לבחור אלמנטים במיקומים לא רציפים:

In [ ]:
# בחירת אלמנטים ספציפיים
gathered = trending_dates_by_channel.select(
    'trending_date',
    # ראשון ואחרון
    pl.col('trending_date').list.gather([0, -1]).alias('first_and_last'),
    
    # ראשון 4 פעמים
    pl.col('trending_date').list.gather([0, 0, 0, 0]).alias('first_repeated'),
    
    # ראשון ו-11 (עם null אם לא קיים)
    pl.col('trending_date').list.gather([0, 10], null_on_oob=True).alias('first_and_11th')
).head()

print("🎯 בחירה מותאמת:")
gathered

### 🎓 תרגיל 3: גישה לאלמנטים

**משימה**: 
1. צור DataFrame שמקבץ את כל הכותרות לפי קטגוריה
2. הצג את 3 הכותרות הראשונות מכל קטגוריה
3. הצג את הכותרת האמצעית (באמצע הרשימה)

In [ ]:
# כתוב את הקוד שלך כאן:
# רמז: list.get(pl.col('title').list.len() // 2)



### 🔹 פעולות נוספות: מיון ודגימה

In [ ]:
# 3 התאריכים האחרונים (לאחר מיון)
recent_dates = trending_dates_by_channel.with_columns(
    pl.col('trending_date')
    .list.sort(descending=True)  # מיין בסדר יורד
    .list.head(3)  # קח 3 ראשונים
    .alias('3_most_recent')
).head()

print("📅 3 התאריכים האחרונים:")
recent_dates

---
<a id="החלת-לוגיקה"></a>
## ⚙️ 4. החלת לוגיקה על אלמנטים (Applying Logic to Elements)

### מהי `list.eval()`?

`list.eval()` מאפשר להחיל **פעולה על כל אלמנט** ברשימה בנפרד.

### תחביר בסיסי:

```python
pl.col('my_list').list.eval(
    פעולה_על_כל_אלמנט
)
```

### הקשרים מיוחדים:

- `pl.element()` - האלמנט הנוכחי
- `pl.first()` - האלמנט הראשון ברשימה
- `pl.last()` - האלמנט האחרון ברשימה
- `pl.col('')` - כל העמודה (השורה הנוכחית)

### הכנת נתונים

In [ ]:
# נצור DataFrame עם רשימות של צפיות וערוצים
agg_df = (
    df
    .group_by('trending_date')
    .agg('views', 'channel_title')
)

print("📊 נתונים מצוברים:")
agg_df.head()

### 🔹 החזרת הרשימה כמות שהיא

בואו נראה את ההבדל בין הקשרים שונים:

In [ ]:
# כל ההקשרים מחזירים את אותה רשימה (כי אין פעולה)
contexts = agg_df.select(
    'views',
    pl.col('views').list.eval(pl.element()).alias('element_context'),
    pl.col('views').list.eval(pl.first()).alias('first_context'),
    pl.col('views').list.eval(pl.last()).alias('last_context'),
    pl.col('views').list.eval(pl.col('')).alias('col_context')
).head(3)

print("🔍 הקשרים שונים:")
contexts

### 🔹 המרה לאותיות גדולות

In [ ]:
# המרת שמות ערוצים לאותיות גדולות
uppercase_channels = (
    agg_df
    .select(
        pl.col('channel_title').list.head(2),
        pl.col('channel_title')
        .list.eval(pl.element().str.to_uppercase())
        .list.head(2)
        .alias('channel_uppercase')
    )
).head()

print("🔤 המרה לאותיות גדולות:")
uppercase_channels

### 💡 **הסבר מפורט**

1. `pl.col('channel_title')` - בוחר את עמודת הערוצים
2. `.list.eval(...)` - החל פעולה על כל אלמנט ברשימה
3. `pl.element()` - האלמנט הנוכחי (שם ערוץ)
4. `.str.to_uppercase()` - המר לאותיות גדולות
5. `.list.head(2)` - קח רק 2 ראשונים (להצגה)

### 🔹 סינון אלמנטים

נוכל לסנן אלמנטים שמקיימים תנאי:

In [ ]:
# השאר רק ערוצים שמכילים את האות A
filtered_channels = uppercase_channels.with_columns(
    pl.col('channel_uppercase')
    .list.eval(
        pl.element().filter(
            pl.element().str.contains('A', literal=True)
        )
    )
).head()

print("🔍 ערוצים עם A:")
filtered_channels

### 🔹 חישוב דירוג (Ranking)

In [ ]:
# דירוג צפיות בתוך כל יום
ranked_views = agg_df.select(
    'trending_date',
    'views',
    pl.col('views')
    .list.eval(
        pl.element().rank('dense', descending=True)
    )
    .alias('views_rank')
).head()

print("🏆 דירוג צפיות:")
ranked_views

### 💡 **הסבר**: מה זה דירוג?

- המספר **1** = הכי הרבה צפיות
- המספר **2** = שני בגודל
- וכך הלאה...

לדוגמה: `[748374, 2418783, 3191434]` → `[3, 2, 1]`

### 🔹 חישובים מורכבים: ההפרש מהמקסימום

In [ ]:
# כמה צפיות חסרות לכל סרטון כדי להיות הכי גבוה?
diff_from_max = ranked_views.select(
    'views',
    pl.col('views')
    .list.eval(
        pl.element().max() - pl.element()
    )
    .alias('gap_to_max')
).head()

print("📊 פער מהמקסימום:")
diff_from_max

### 🔹 פעולות על קבוצות (Set Operations)

In [ ]:
# נשווה את הדירוגים של 2 הראשונים מול 2 האחרונים
set_operations = ranked_views.with_columns(
    pl.col('views_rank').list.slice(0, 2).alias('top_2_ranks'),
    pl.col('views_rank').list.slice(-2, 2).alias('bottom_2_ranks')
).select(
    'top_2_ranks',
    'bottom_2_ranks',
    # חיתוך (מה משותף)
    pl.col('top_2_ranks').list.set_intersection('bottom_2_ranks').alias('intersection'),
    # איחוד (כל הערכים הייחודיים)
    pl.col('top_2_ranks').list.set_union('bottom_2_ranks').alias('union'),
    # הפרש (מה ב-A אבל לא ב-B)
    pl.col('top_2_ranks').list.set_difference('bottom_2_ranks').alias('difference')
).head()

print("🔀 פעולות קבוצות:")
set_operations

### 🎓 תרגיל 4: החלת לוגיקה

**משימה**:
1. צור רשימה של מספרי הלייקים לפי תאריך
2. חשב עבור כל סרטון: `likes / (likes + dislikes)` - אחוז הלייקים
3. מצא את הממוצע של אחוז הלייקים בכל יום

In [ ]:
# כתוב את הקוד שלך כאן:



### 🔹 פיצול רשימות (Explode)

לפעמים נרצה "לפרוס" רשימה - להפוך כל אלמנט לשורה נפרדת:

In [ ]:
# דוגמה: פריסת רשימת הצפיות
exploded_example = (
    ranked_views
    .head(2)  # קח רק 2 תאריכים
    .select('trending_date', 'views', 'views_rank')
    .explode('views', 'views_rank')  # פרוס את הרשימות
)

print("💥 לפני explode:")
print(ranked_views.head(2))
print("\n💥 אחרי explode:")
print(exploded_example)

---
<a id="עבודה-עם-structs"></a>
## 🏗️ 5. עבודה עם Structs ו-JSON

### מהו Struct?

**Struct** הוא מבנה נתונים שמכיל מספר **שדות** (fields), כמו dictionary ב-Python.

**דוגמה**:
```python
{
    'name': 'John',
    'age': 30,
    'city': 'Tel Aviv'
}
```

### למה להשתמש ב-Structs?

✅ **ארגון טוב יותר**: מידע קשור נשמר ביחד  
✅ **יעילות**: פחות עמודות, יותר מבנה  
✅ **עבודה עם JSON**: המרה קלה מ/אל JSON

### טעינת נתוני JSON

In [ ]:
# טעינת נתוני Google Analytics בפורמט JSON
# נתונים אלו מכילים structs מקוננים

ga_df = pl.read_json('../data/ga_20170801.json', infer_schema_length=10000)

# נבחר כמה עמודות רלוונטיות
cols = ['visitId', 'date', 'totals', 'trafficSource', 'customDimensions', 'channelGrouping']
ga_df = ga_df.select(cols)

print(f"📊 מספר ביקורים: {ga_df.height:,}")
print(f"\n🔍 5 ביקורים ראשונים:")
ga_df.head()

### בחינת הטיפוסים

In [ ]:
# בואו נראה את טיפוסי העמודות
print("📋 טיפוסי העמודות:")
for col in ga_df.columns:
    dtype = ga_df[col].dtype
    print(f"  {col}: {dtype}")

### 🔹 יצירת Struct מעמודות קיימות

In [ ]:
# נצור struct מ-3 עמודות
with_struct = ga_df.with_columns(
    pl.struct('visitId', 'date', 'channelGrouping').alias('visit_info')
)

print("🏗️ Struct שנוצר:")
with_struct.select('visitId', 'date', 'channelGrouping', 'visit_info').head()

### 💡 **מבנה ה-Struct**

העמודה `visit_info` מכילה struct עם 3 שדות:
- `visitId`
- `date`
- `channelGrouping`

כל תא ב-`visit_info` הוא מבנה שמכיל את 3 הערכים!

### 🔹 פתיחת Struct (Unnesting)

In [ ]:
# נפרק את ה-struct חזרה לעמודות
unnested = (
    with_struct
    .select('visit_info')
    .unnest('visit_info')
).head()

print("📤 Struct שנפרק:")
unnested

### 🔹 גישה לשדה ספציפי ב-Struct

In [ ]:
# גישה לשדה ספציפי
field_access = with_struct.select(
    'visit_info',
    pl.col('visit_info').struct.field('channelGrouping').alias('channel')
).head()

print("🎯 גישה לשדה:")
field_access

### 🔹 שינוי שמות שדות

In [ ]:
# שינוי שמות השדות ב-struct
renamed = with_struct.select(
    'visit_info',
    pl.col('visit_info')
    .struct.rename_fields(['id', 'date_str', 'channel'])
    .alias('visit_info_renamed')
).head()

print("✏️ שמות שדות חדשים:")
renamed

### 🔹 עבודה עם Struct קיים

In [ ]:
# נפרק את ה-struct של trafficSource
traffic_unnested = (
    ga_df
    .select('trafficSource')
    .unnest('trafficSource')
).head()

print("🚦 מקור תעבורה מפורק:")
traffic_unnested

### 🔹 שילוב Structs וגישה לשדות

In [ ]:
# נשלב ערוץ עם מקור התעבורה
combined = ga_df.select(
    pl.struct(
        pl.col('channelGrouping'),
        pl.col('trafficSource').struct.field('source')
    )
    .unique()
    .alias('channel_and_source')
).unnest('channel_and_source').sort('channelGrouping', 'source')

print("🔀 שילוב ערוץ ומקור:")
combined

### 🔹 המרה ל-JSON וחזרה

In [ ]:
# המרת struct ל-JSON
json_conversion = (
    ga_df
    .select(
        # המרה ל-JSON string
        pl.col('totals').struct.json_encode().alias('totals_json'),
        # המרה חזרה ל-struct
        pl.col('totals').struct.json_encode().str.json_decode(dtype=pl.Struct).alias('totals_decoded')

    )
).head()

print("🔄 המרה ל-JSON וחזרה:")
json_conversion

### 💡 **מתי להשתמש ב-JSON encoding/decoding?**

- ✅ שמירת נתונים לקובץ
- ✅ העברת נתונים בין מערכות
- ✅ אחסון במסד נתונים
- ✅ API requests/responses

### 🔹 המרת רשימה ל-Struct

In [ ]:
# נקבץ נתונים ונהפוך רשימה ל-struct
list_to_struct = (
    ga_df
    .group_by('channelGrouping')
    .agg([
        pl.col('visitId'),
        pl.col('visitId').len().alias('numVisits')
        ])
    .sort('numVisits')
    .with_columns(
        # המרת הרשימה ל-struct (כל אלמנט הופך לשדה)
        pl.col('visitId').list.to_struct(upper_bound=10).alias('visits_struct')
    )
)

print("📊 רשימה שהומרה ל-struct:")
list_to_struct

### 🎓 תרגיל 5: עבודה עם Structs

**משימה**:
1. צור struct שמכיל: `video_id`, `title`, `views`, `likes`
2. המר אותו ל-JSON
3. פרק את ה-struct והצג רק את `title` ו-`views`

In [ ]:
# כתוב את הקוד שלך כאן:



---
<a id="תרגילים"></a>
## 🎯 תרגילים ואתגרים נוספים

### תרגיל 6: ניתוח מתקדם של תגיות

**משימה מורכבת**:
1. פצל את עמודת `tags` לרשימה
2. מצא את 10 התגיות הכי נפוצות
3. עבור כל תגית, חשב את ממוצע הצפיות של סרטונים עם התגית הזו
4. הצג גרף (אופציונלי)

In [ ]:
# כתוב את הקוד שלך כאן:
# רמזים:
# 1. str.split('|')
# 2. explode()
# 3. group_by() + agg()
# 4. value_counts()



### תרגיל 7: מציאת סרטונים "ויראליים"

**משימה**:
1. עבור כל ערוץ, צור רשימה של מספרי הצפיות
2. חשב את הסטיית התקן של הצפיות
3. מצא סרטונים שהצפיות שלהם גבוהות פי 3+ מהממוצע
4. הצג את הערוצים עם הכי הרבה סרטונים "ויראליים"

In [ ]:
# כתוב את הקוד שלך כאן:



### תרגיל 8: ניתוח זמני (Temporal Analysis)

**משימה**:
1. לכל ערוץ, צור רשימה של התאריכים שבהם היה טרנדי
2. חשב את ההפרשים בין תאריכים עוקבים
3. מצא ערוצים שהיו טרנדיים באופן עקבי (פער קטן בין תאריכים)

In [ ]:
# כתוב את הקוד שלך כאן:



---
<a id="סיכום"></a>
## 📚 סיכום ומשאבים נוספים

### סיכום המושגים העיקריים

| נושא | פעולות עיקריות | דוגמה |
|------|----------------|-------|
| **יצירת רשימות** | `str.split()`, `group_by().agg()`, `concat_list()` | `pl.col('tags').str.split('│')` |
| **צבירה** | `list.min/max/mean/sum/len()` | `pl.col('views').list.mean()` |
| **גישה לאלמנטים** | `list.first/last/get/head/tail()` | `pl.col('dates').list.first()` |
| **חיתוך** | `list.slice()`, `list.gather()` | `pl.col('items').list.slice(0,3)` |
| **החלת לוגיקה** | `list.eval()` | `pl.col('lst').list.eval(pl.element()*2)` |
| **Structs** | `struct()`, `unnest()`, `struct.field()` | `pl.struct('a','b','c')` |
| **JSON** | `struct.json_encode/decode()` | `pl.col('data').struct.json_encode()` |

### טיפים וטריקים חשובים

#### 1. ⚡ ביצועים
```python
# ✅ טוב - פעולה אחת
df.with_columns(
    pl.col('tags').str.split('|').alias('tags_list')
)

# ❌ פחות טוב - פעולות מרובות
df.with_columns(
    pl.col('tags').str.split('|')
).with_columns(
    pl.col('tags').list.len()
)
```

#### 2. 🛡️ טיפול ב-nulls
```python
# תמיד השתמש ב-null_on_oob
pl.col('my_list').list.get(10, null_on_oob=True)
```

#### 3. 🔍 דיבאג
```python
# השתמש ב-glimpse() לבדוק טיפוסים
df.glimpse()

# הצג מספר מוגבל של אלמנטים
pl.col('my_list').list.head(3)
```

#### 4. 🎨 קריאות קוד
```python
# ✅ קריא - שלבים ברורים
(
    df
    .group_by('date')
    .agg('views')
    .with_columns(
        pl.col('views').list.mean().alias('avg_views')
    )
)
```

### מלכודות נפוצות (Common Pitfalls)

#### ❌ לא להשתמש ב-eval בלי צורך
```python
# ❌ מיותר
pl.col('views').list.eval(pl.element().sum())

# ✅ פשוט יותר
pl.col('views').list.sum()
```

#### ❌ לשכוח null_on_oob
```python
# ❌ עלול לזרוק שגיאה
pl.col('items').list.get(100)

# ✅ בטוח
pl.col('items').list.get(100, null_on_oob=True)
```

#### ❌ ערבוב explode עם agg
```python
# ❌ לא עובד
df.explode('tags').group_by('date').agg('tags')

# ✅ נכון
df.group_by('date').agg('tags')
```

### משאבים נוספים

#### 📖 תיעוד רשמי
- [Polars User Guide](https://pola-rs.github.io/polars/user-guide/)
- [API Reference](https://pola-rs.github.io/polars/py-polars/html/reference/)
- [List Functions](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/list.html)

#### 🎓 קורסים ומדריכים
- [Polars Tutorial on Real Python](https://realpython.com/polars-python/)
- [Modern Polars (GitHub)](https://github.com/pola-rs/polars-book)

#### 💬 קהילה
- [Polars Discord Server](https://discord.gg/4UfP5cfBE7)
- [Stack Overflow - polars tag](https://stackoverflow.com/questions/tagged/polars)
- [GitHub Discussions](https://github.com/pola-rs/polars/discussions)

#### 🎬 וידאו
- [Polars YouTube Channel](https://www.youtube.com/@polarsDataFrame)

---

### 🎉 סיימתם!

**מזל טוב!** עכשיו אתם יודעים:
- ✅ ליצור ולנהל רשימות ב-Polars
- ✅ לצבור ולנתח נתונים מקוננים
- ✅ לגשת ולבחור אלמנטים ברשימות
- ✅ להחיל פעולות מורכבות על רשימות
- ✅ לעבוד עם Structs ו-JSON

**הצעד הבא**: נסו להשתמש בטכניקות אלו בפרויקט אמיתי שלכם! 🚀

---

### 📝 משוב

מצאתם שגיאה? יש הצעות לשיפור? אנא פנו אלינו!

**בהצלחה בדרך שלכם עם Polars!** 🐻‍❄️